## predict

In [1]:
# UltralyticsのYOLOv8を使用して画像推論を実行
from ultralytics import YOLO
import os 

model = YOLO(r"D:\prog_py\Yolo_trainer\runs\train\train15\weights\best.pt")

In [ ]:
# 指定されたパラメータで画像推論を実行
print("推論を開始します...")

source_path = r"D:\prog_py\yolo12_detect\detect_images"
if os.path.exists(source_path):
    print("🔍 画像サイズ（32の倍数, 1500まで）で推論を実行しています...")

    # 32の倍数で512から1248まで
    grid_imgsz = list(range(512, 1248, 32))

    for imgsz in grid_imgsz:
        print(f"\n--- 画像サイズ: {imgsz} で推論中 ---")
        # 実験名を画像サイズごとに分けて保存することで各サイズごとに結果が見やすく、すべての画像サイズで推論できるようにする
        results = model.predict(
            source=source_path,           # 画像フォルダ: detect_images
            imgsz=imgsz,                  # 画像サイズを動的に変更
            save_txt=True,                # テキストファイル保存: True
            line_width=1,                 # バウンディングボックスの線の太さ: 1
            save=True,                    # 推論結果の画像を保存
            conf=0.25,                    # 信頼度のしきい値
            device='0',                   # デバイス（GPUが利用可能な場合は'0'に変更）
            project='runs/detect',        # 結果保存ディレクトリ
            name=f'yolo_detection_{imgsz}',# 画像サイズごとにディレクトリ分け
            agnostic_nms=True,
        )

print("✓ 推論が完了しました！")
print(f"結果は runs/detect/yolo_detection_＜imgsz＞ ディレクトリに保存されました")


## predict single imgsz

In [3]:
model.predict(source= r"D:\prog_py\yolo12_detect\detect_p2pnet_images", 
            save=True,
            imgsz=1500, 
            conf=0.5,
            save_txt=True,                # テキストファイル保存: True
            line_width=1,
            agnostic_nms=True  
            )


WARNING imgsz=[1500] must be multiple of max stride 32, updating to [1504]
image 1/13 D:\prog_py\yolo12_detect\detect_p2pnet_images\Image_000003.jpg: 1504x1376 74 items, 272.9ms
image 2/13 D:\prog_py\yolo12_detect\detect_p2pnet_images\Image_000118.jpg: 1504x896 72 items, 107.5ms
image 3/13 D:\prog_py\yolo12_detect\detect_p2pnet_images\Image_000188.jpg: 1504x864 64 items, 98.7ms
image 4/13 D:\prog_py\yolo12_detect\detect_p2pnet_images\Image_000473.jpg: 1504x832 64 items, 111.9ms
image 5/13 D:\prog_py\yolo12_detect\detect_p2pnet_images\Image_000565.jpg: 1504x608 102 items, 110.0ms
image 6/13 D:\prog_py\yolo12_detect\detect_p2pnet_images\Image_000896.jpg: 1504x672 63 items, 100.1ms
image 7/13 D:\prog_py\yolo12_detect\detect_p2pnet_images\Image_001075.jpg: 1504x1088 62 items, 95.4ms
image 8/13 D:\prog_py\yolo12_detect\detect_p2pnet_images\Image_001845.jpg: 1504x576 62 items, 113.8ms
image 9/13 D:\prog_py\yolo12_detect\detect_p2pnet_images\Image_001851.jpg: 1504x736 34 items, 96.4ms
image 

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: ultralytics.engine.results.Masks object
 names: {0: 'item'}
 obb: None
 orig_img: array([[[ 85, 115, 120],
         [ 89, 119, 124],
         [ 97, 126, 130],
         ...,
         [118, 156, 161],
         [101, 136, 140],
         [ 98, 130, 135]],
 
        [[ 84, 113, 120],
         [ 92, 121, 128],
         [103, 133, 138],
         ...,
         [126, 162, 168],
         [112, 147, 151],
         [ 99, 131, 136]],
 
        [[ 97, 128, 137],
         [105, 136, 145],
         [118, 149, 158],
         ...,
         [130, 164, 170],
         [126, 158, 163],
         [ 99, 129, 134]],
 
        ...,
 
        [[ 92, 130, 162],
         [108, 146, 178],
         [125, 163, 195],
         ...,
         [190, 198, 205],
         [179, 189, 196],
         [172, 182, 189]],
 
        [[127, 165, 197],
         [128, 166, 198],
         [135, 173, 205],

## VAL

In [7]:

model = YOLO(r"D:\prog_py\Yolo_trainer\runs\train\train15\weights\best.pt")
# dataset.yamlのパス
dataset_yaml_path = r"D:\prog_py\Yolo_trainer\ultralytic_series\data\all_datasets\YOLODataset\dataset.yaml"

# 画像サイズ（imgsz）はmax stride 32の倍数でグリッドサーチ
if os.path.exists(dataset_yaml_path):
    print("🔍 画像サイズ（32の倍数, 1500まで）で検証を実行しています...")

    # 32の倍数で500から1504まで
    grid_imgsz = list(range(512, 1248, 32))

    for imgsz in grid_imgsz:
        print(f"\n--- 画像サイズ: {imgsz} で検証中 ---")
        validation_results = model.val(
            data=dataset_yaml_path,
            imgsz=imgsz,
            plots=True,
            batch=8,
            save_json=True,
            device='0',
            project='runs/val',
            name=f'val_imgs_{imgsz}',
            verbose=True,
        )


else:
    print("❌ データセット設定ファイルが見つからないため、検証をスキップします。")


🔍 画像サイズ（32の倍数, 1500まで）で検証を実行しています...

--- 画像サイズ: 512 で検証中 ---
Ultralytics 8.3.223  Python-3.12.6 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
YOLOv12n-seg summary (fused): 172 layers, 2,779,387 parameters, 0 gradients, 9.1 GFLOPs
val: Fast image access  (ping: 0.10.0 ms, read: 975.8518.9 MB/s, size: 881.3 KB)
val: Scanning D:\prog_py\Yolo_trainer\ultralytic_series\data\all_datasets\YOLODataset\labels\val.cache... 11 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 11/11 11.0Kit/s 0.0s
val: D:\prog_py\Yolo_trainer\ultralytic_series\data\all_datasets\YOLODataset\images\val\045_06_f.jpg: 2 duplicate labels removed
val: D:\prog_py\Yolo_trainer\ultralytic_series\data\all_datasets\YOLODataset\images\val\057_06_b.jpg: 2 duplicate labels removed
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 2/2 0.0it/s 1:13<3:01
                   all         11       1001     

KeyboardInterrupt: 

## single val

##　出力変換

In [9]:
model = YOLO(r"D:\prog_py\Yolo_trainer\runs\train\train15\weights\best.pt")
# dataset.yamlのパス
dataset_yaml_path = r"D:\prog_py\Yolo_trainer\ultralytic_series\data\all_datasets\YOLODataset\dataset.yaml"
imgsz = 1400

validation_results = model.val(
            data=dataset_yaml_path,
            imgsz=imgsz,
            plots=True,
            batch=8,
            save_json=True,
            device='0',
            project='runs/val',
            name=f'val_imgs_{imgsz}',
            verbose=True,
        )

WARNING imgsz=[1400] must be multiple of max stride 32, updating to [1408]
Ultralytics 8.3.223  Python-3.12.6 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
YOLOv12n-seg summary (fused): 172 layers, 2,779,387 parameters, 0 gradients, 9.1 GFLOPs
val: Fast image access  (ping: 0.10.0 ms, read: 780.6332.2 MB/s, size: 1129.8 KB)
val: Scanning D:\prog_py\Yolo_trainer\ultralytic_series\data\all_datasets\YOLODataset\labels\val.cache... 11 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 11/11  0.0s
val: D:\prog_py\Yolo_trainer\ultralytic_series\data\all_datasets\YOLODataset\images\val\045_06_f.jpg: 2 duplicate labels removed
val: D:\prog_py\Yolo_trainer\ultralytic_series\data\all_datasets\YOLODataset\images\val\057_06_b.jpg: 2 duplicate labels removed
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 2/2 0.0it/s 2:487:58
                   all         11       1001 

In [ ]:
from ultralytics import YOLO

# Load a model

model = YOLO(r"D:\prog_py\Yolo_trainer\runs\train\train9\weights\best.pt")  # load a custom trained model

# Export the model
model.export(format="onnx",
             imgsz=800)

## auto label

In [10]:
from ultralytics.data.annotator import auto_annotate

auto_annotate(data=r"E:\data\pods\pod2024\Datase_2024_pod_insta360_backup\test\images",
              det_model=r"D:\prog_py\Yolo_trainer\runs\train\all_class_pod\weights\best.pt",
              imgsz=2000,
              sam_model="sam2.1_l.pt")


WARNING imgsz=[2000] must be multiple of max stride 32, updating to [2016]
image 1/23 E:\data\pods\pod2024\Datase_2024_pod_insta360_backup\test\images\024_04_b.jpg: 2016x992 7 0spps, 2 1spps, 45 1spp_occs, 24 2spps, 42 2spp_occs, 2 3spps, 6 pod_occs, 132.7ms
image 2/23 E:\data\pods\pod2024\Datase_2024_pod_insta360_backup\test\images\024_04_f.jpg: 2016x960 1 0spp, 3 1spps, 24 1spp_occs, 14 2spps, 32 2spp_occs, 3 3spp_occs, 2 pod_occs, 3518.6ms
image 3/23 E:\data\pods\pod2024\Datase_2024_pod_insta360_backup\test\images\043_03_b.jpg: 2016x1216 2 0spps, 1 1spp, 16 1spp_occs, 14 2spps, 20 2spp_occs, 2 3spps, 2 3spp_occs, 6 pod_occs, 2549.8ms
image 4/23 E:\data\pods\pod2024\Datase_2024_pod_insta360_backup\test\images\043_3_f.jpg: 2016x1056 3 0spps, 2 1spps, 18 1spp_occs, 10 2spps, 23 2spp_occs, 2 3spp_occs, 4 pod_occs, 2519.2ms
image 5/23 E:\data\pods\pod2024\Datase_2024_pod_insta360_backup\test\images\045_05_b.jpg: 2016x1504 1 0spp, 1 1spp, 7 1spp_occs, 7 2spps, 10 2spp_occs, 1 pod_occ, 62